In [1]:
grid = []

chars = set()

with open("./data/day23.txt") as f:
    while line := f.readline():
        line = line.rstrip()
        grid.append(list(line))
        for c in line:
            chars.add(c)

rows, cols = len(grid), len(grid[0])
chars

{'#', '.', '>', 'v'}

# Part 1

Simple DFS to enumerate all paths. Note that the example and input data actually don't contain the characters `^` and `<`.

In [2]:
def get_directions(row, col, path):
    if grid[row][col] == '>':
        return [(row, col+1)] if (row, col+1) not in path else []
    if grid[row][col] == 'v':
        return [(row+1, col)] if (row+1, col) not in path else []
    res = []
    for i, j in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
        if (row+i, col+j) not in path and row+i in range(rows) and col+j in range(cols) and grid[row+i][col+j] != '#':
            res.append((row+i, col+j))
    return res

In [3]:
q = [(0, 1, set([(0, 1),]))]
res = 0
while q:
    row, col, path = q.pop()
    if row == rows-1:
        res = max(res, len(path))
        continue
    for new_row, new_col in get_directions(row, col, path):
        new_path = path.copy()
        new_path.add((new_row, new_col))
        q.append((new_row, new_col, new_path))
res-1

2406

# Part 2

We can now ignore the slopes. Enumerating all paths like in part 1 takes too long with python, at least for my taste, but people did say on reddit that they did simply run their part 1 solution for part 2 with more performant languages and it only took a few minutes.

To make the pathfinding more efficient, we'll find the forks in the paths and create a graph from the forks and immediate connections between forks. We will also only keep the longest direct path between any two forks which might save us some irrelevant paths.

In [4]:
def get_directions2(row, col, path):
    res = []
    for i, j in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
        if (row+i, col+j) not in path and row+i in range(rows) and col+j in range(cols) and grid[row+i][col+j] != '#':
            res.append((row+i, col+j))
    return res

In [5]:
def is_fork(row, col):
    if grid[row][col] == '#':
        return False
    res = 0
    for i, j in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
        if row+i in range(rows) and col+j in range(cols) and grid[row+i][col+j] != '#':
            res += 1
    return res > 2

In [6]:
forks = []
forks.append((0, 1))
forks.append((rows-1, cols-2))
for row in range(rows):
    for col in range(cols):
        if is_fork(row, col):
            forks.append((row, col))
len(forks)

36

In [7]:
from collections import deque

In [8]:
fork_graph = {fork:{} for fork in forks}
for r, c in forks:
    q = deque()
    q.append((r, c, set([(r, c),])))
    while q:
        row, col, path = q.pop()
        if (row, col) != (r, c) and (row, col) in forks:
            fork_graph[(r, c)][(row, col)] = len(path)-1
            continue
        ls = get_directions2(row, col, path)
        for new_row, new_col in ls:
            if len(ls) > 1:
                new_path = path.copy()
            else:
                new_path = path
            new_path.add((new_row, new_col))
            q.appendleft((new_row, new_col, new_path))
for i, (k, v) in enumerate(fork_graph.items()):
    if i >= 10:
        print('...')
        print('...')
        break
    print(f'Fork {k} immediate neighbors and longest paths to: {v}')

Fork (0, 1) immediate neighbors and longest paths to: {(13, 11): 155}
Fork (140, 139) immediate neighbors and longest paths to: {(129, 123): 135}
Fork (5, 113) immediate neighbors and longest paths to: {(37, 113): 208, (13, 83): 278, (43, 129): 434}
Fork (13, 11) immediate neighbors and longest paths to: {(33, 15): 116, (0, 1): 155, (19, 43): 290}
Fork (13, 83) immediate neighbors and longest paths to: {(35, 89): 120, (17, 59): 236, (5, 113): 278}
Fork (17, 59) immediate neighbors and longest paths to: {(29, 61): 58, (19, 43): 134, (13, 83): 236}
Fork (19, 43) immediate neighbors and longest paths to: {(33, 31): 82, (17, 59): 134, (13, 11): 290}
Fork (29, 61) immediate neighbors and longest paths to: {(17, 59): 58, (35, 89): 142, (33, 31): 146, (63, 53): 222}
Fork (33, 15) immediate neighbors and longest paths to: {(33, 31): 64, (13, 11): 116, (67, 13): 316}
Fork (33, 31) immediate neighbors and longest paths to: {(33, 15): 64, (19, 43): 82, (29, 61): 146, (65, 29): 218}
...
...


Since we need to enumerate all paths in this graph, DFS and BFS both work. There is a difference in how we can keep track of the current path between the two. With BFS, (I think) we need to store the entire path for each path separately while we process them, but with DFS we can keep track of visited nodes globally by using backtracking. The problem with BFS requiring us to store each path is not just that we need to copy the entire previous path when we split our path, but simply the fact that we need to store all paths until they reach the end. This hits the limit of RAM on my machine which obviously slows the algorithm down to a crawl.

So we cannot do a normal BFS. I still want to test how inefficient it is to need to copy the entire previous path whenever our path splits to two or more paths, so let's try a DFS with a stack implementation where we keep track of the entire paths like we would in BFS.

DFS without recursion. Keep track of current path in a set. Requires copying the entire path every time we move forward.

In [9]:
import time

In [10]:
starttime = time.time()

q = [(0, 1, set([(0, 1),]), 0)]
res = 0
while q:
    row, col, path, d = q.pop()
    if row == rows-1:
        res = max(res, d)
        continue
    adj = fork_graph[(row, col)]
    for (new_row, new_col), dist in adj.items():
        if (new_row, new_col) in path:
            continue
        new_path = path.copy()
        new_path.add((new_row, new_col))
        q.append((new_row, new_col, new_path, d+dist))
print(res)

print(f'Runtime {time.time()-starttime:.1f}s')

6630
Runtime 31.1s


DFS using recursion. Keep track of current path in a dictionary mapping visited forks to `True`. clear visited status with backtracking.

In [11]:
starttime = time.time()

forks_visited = {fork:False for fork in forks}
forks_visited[(0, 1)] = True
results = []
def dfs(row, col, d):
    global forks_visited, results
    if row == rows-1:
        results.append(d)
        return
    adj = fork_graph[(row, col)]
    for (new_row, new_col), dist in adj.items():
        if forks_visited[(new_row, new_col)]:
            continue
        forks_visited[(new_row, new_col)] = True
        dfs(new_row, new_col, d+dist)
        forks_visited[(new_row, new_col)] = False
dfs(0, 1, 0)
print(max(results))

print(f'Runtime {time.time()-starttime:.1f}s')

6630
Runtime 15.5s
